# Clustering Exercises - Data Wrangling

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

from env import host, user, password
import os

#import acquire
#import summarize
#import prepare

## acquire & summarize

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.
    * Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
    * Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
    * Only include properties that include a latitude and longitude value.

2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

### acquire

In [2]:
# copy and pasted from my work on the regression project and then modified

def get_connection(db, user=user, host=host, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

def get_zillow_data():   
    sql_query = '''
                SELECT *
                FROM properties_2017
                JOIN(SELECT parcelid, logerror, max(transactiondate) AS lasttransactiondate
                FROM predictions_2017
                GROUP BY parcelid, logerror
                ) AS predictions USING (parcelid)
                LEFT JOIN airconditioningtype USING (airconditioningtypeid)
                LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
                LEFT JOIN buildingclasstype USING (buildingclasstypeid)
                LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
                LEFT JOIN propertylandusetype USING (propertylandusetypeid)
                LEFT JOIN storytype USING (storytypeid)
                LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
                LEFT JOIN unique_properties USING (parcelid)
                WHERE latitude is not null AND longitude is not null
                AND propertylandusetypeid = '261' OR '262' OR '263' OR '264' OR '268' OR '273' OR '274' OR '275' OR '276' OR '279';
                '''
    filename = "zillow_df.csv"
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
        return df
    else:
        # read the SQL query into a dataframe
        df = pd.read_sql(sql_query, get_connection('zillow'))
        # cache the dataframe
        df.to_csv(filename)
        
        # return the dataframe
        return df

In [3]:
df = get_zillow_data()
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,censustractandblock,logerror,lasttransactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,10711855,nan,nan,261.00,2.00,nan,nan,nan,1087254,nan,...,"60,371,132,321,007.00",-0.01,2017-07-07,None,None,None,Central,Single Family Residential,None,None
1,10711877,nan,nan,261.00,2.00,nan,nan,1.00,1072280,nan,...,"60,371,132,321,007.00",0.02,2017-08-29,Central,None,None,Central,Single Family Residential,None,None
2,10711888,nan,nan,261.00,2.00,nan,nan,1.00,1340933,nan,...,"60,371,132,321,007.00",0.08,2017-04-04,Central,None,None,Central,Single Family Residential,None,None
3,10711910,nan,nan,261.00,2.00,nan,nan,nan,1878109,nan,...,"60,371,132,321,008.00",-0.04,2017-03-17,None,None,None,Central,Single Family Residential,None,None
4,10711923,nan,nan,261.00,2.00,nan,nan,nan,2190858,nan,...,"60,371,132,321,008.00",-0.01,2017-03-24,None,None,None,Central,Single Family Residential,None,None


In [4]:
df.columns

Index(['parcelid', 'typeconstructiontypeid', 'storytypeid',
       'propertylandusetypeid', 'heatingorsystemtypeid', 'buildingclasstypeid',
       'architecturalstyletypeid', 'airconditioningtypeid', 'id',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'decktypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'latitude',
       'longitude', 'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
       'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertyzoningdesc',
       'rawcensustractandblock', 'regionidcity', 'regionidcounty',
       'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequarterbathnbr',
       'unitcnt', 'yardbuildingsqft17',

### summarize